In [2]:
#install.packages("readxl")
library(tidyverse)
library(rvest)
library(readxl)

─ Attaching packages ──────────────────── tidyverse 1.3.1 ─

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.5     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.0.2     ✔ forcats 0.5.1

─ Conflicts ───────────────────── tidyverse_conflicts() ─
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


载入程辑包：‘rvest’


The following object is masked from ‘package:readr’:

    guess_encoding




In [3]:
#first we get all xlsx file links from url
mental_health_data_urls_all_years <- 'https://www.health.govt.nz/our-work/mental-health-and-addiction/mental-health-and-addiction-monitoring-reporting-and-data' %>% 
                                        read_html() %>%
                                        html_element('#node-12245 > div > div > div > ul:nth-child(19)') %>%
                                        html_elements("a") %>%
                                        html_attrs() %>% 
                                        str_c('https://www.health.govt.nz/', .)

#function is to download xlsx from health url 
download_xlsx_file <- function(heath_url, file_name){
                        download_ur <- heath_url %>% 
                            read_html() %>%
                            html_element("#block-system-main > div > div:nth-child(2) > div.panel-panel.right.col-xs-12.col-sm-12.col-md-3.col-lg-3 > div.panel-pane.pane-views.pane-file-attachments-revisioning.pane-downloads-right.standard-block.even.hidden-xs.hidden-sm > div > div > div > div > ul > li > span > a") %>% 
                            html_attr("href") 
                        download.file(url = download_ur, destfile = file_name)}

#iteral through every year from 2014 to 2019
start_year = 2014
for (url_yearly_health in mental_health_data_urls_all_years){
    file_name = str_c('mental_health', start_year) %>% 
                    str_c('.xlsx')
    start_year = start_year + 1
    download_xlsx_file(url_yearly_health, file_name)
    }

In [193]:
#list all download file names
file_name = c('mental_health2020.xlsx','mental_health2019.xlsx','mental_health2018.xlsx','mental_health2017.xlsx','mental_health2016.xlsx','mental_health2015.xlsx','mental_health2014.xlsx')

In [194]:
#get all col names for table use after
age_group_col_names <-file_name[1] %>% read_excel(sheet = "table1&2", range = cell_rows(4:4)) %>% colnames()
add_sex_totl_age_group_col_names <- c('sex', "Total", age_group_col_names)


In [92]:
#two different way get first table of total group data
Ethnic_group_total_df <- file_name[1] %>% read_excel(sheet = "table1&2", range = "B5:U7", col_names = add_sex_totl_age_group_col_names) #range = cell_rows(3:19), 
Ethnic_group_total_df  #get from part of xlsx

all_data <- file_name[1] %>% read_excel(sheet = "table1&2", skip=1)
df_total_first_table <- all_data[2:4,2:21]
colnames(df_total_first_table) <- add_sex_totl_age_group_col_names  #get frome whole table  which i think it can be uesd for perstentation show different way get data and compare somth like that
df_total_first_table

sex,Total,0-,5-,10-,15-,20-,25-,30-,35-,40-,45-,50-,55-,60-,65-,70-,75-,80-,85+
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Total,158158,1415,6764,14041,20401,16614,14168,12932,12345,13133,11307,9707,6760,4636,3434,2817,2559,2417,2708
Male,83267,920,4730,7135,10001,9345,7750,6855,6651,7071,6088,5071,3507,2308,1546,1247,1091,961,990
Female,74891,495,2034,6906,10400,7269,6418,6077,5694,6062,5219,4636,3253,2328,1888,1570,1468,1456,1718


New names:
* `` -> ...5
* `` -> ...6
* `` -> ...7
* `` -> ...8
* `` -> ...9
* ...



sex,Total,0-,5-,10-,15-,20-,25-,30-,35-,40-,45-,50-,55-,60-,65-,70-,75-,80-,85+
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Total,158158,1415,6764,14041,20401,16614,14168,12932,12345,13133,11307,9707,6760,4636,3434,2817,2559,2417,2708
Male,83267,920,4730,7135,10001,9345,7750,6855,6651,7071,6088,5071,3507,2308,1546,1247,1091,961,990
Female,74891,495,2034,6906,10400,7269,6418,6077,5694,6062,5219,4636,3253,2328,1888,1570,1468,1456,1718


In [94]:
#first table, maori df
df_maori_first_table <- all_data[5:7,2:21]
colnames(df_maori_first_table) <- add_sex_totl_age_group_col_names 
#first table, Pacific df
df_Pacific_first_table <- all_data[8:10,2:21]
colnames(df_Pacific_first_table) <- add_sex_totl_age_group_col_names 
#first table, Asian df
df_Asian_first_table <- all_data[11:13,2:21]
colnames(df_Asian_first_table) <- add_sex_totl_age_group_col_names 
#first table, other df
df_other_first_table <- all_data[14:16,2:21]
colnames(df_other_first_table) <- add_sex_totl_age_group_col_names 

# df_Pacific_first_table
# df_maori_first_table
# df_Asian_first_table
# df_other_first_table

sex,Total,0-,5-,10-,15-,20-,25-,30-,35-,40-,45-,50-,55-,60-,65-,70-,75-,80-,85+
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Total,102644,813,4488,8107,12405,9522,7978,7602,7594,8470,7584,6897,5140,3734,2909,2400,2224,2199,2578
Male,52083,528,3081,4053,5536,5094,4180,3807,3958,4491,4035,3631,2705,1853,1301,1072,940,875,943
Female,50561,285,1407,4054,6869,4428,3798,3795,3636,3979,3549,3266,2435,1881,1608,1328,1284,1324,1635


In [95]:
#table name for rate df
add_sex_rate_age_group_col_names <- c('sex', "Crude_rate", age_group_col_names, 'ASR')

In [130]:
#second table tolal df
df_tolal_second_table <- all_data[24:26,2:22]
colnames(df_tolal_second_table) <- add_sex_rate_age_group_col_names 

#second table, maori df
df_maori_second_table <- all_data[27:29,2:22]
colnames(df_maori_second_table) <- add_sex_rate_age_group_col_names 
#second table, Pacific df
df_Pacific_second_table <- all_data[30:32,2:22]
colnames(df_Pacific_second_table) <- add_sex_rate_age_group_col_names 
#second table, Asian df
df_Asian_second_table <- all_data[33:35,2:22]
colnames(df_Asian_second_table) <- add_sex_rate_age_group_col_names 
#second table, other df
df_other_second_table <- all_data[36:38,2:22]
colnames(df_other_second_table) <- add_sex_rate_age_group_col_names 

# df_other_second_table
# df_maori_second_table
# df_Pacific_second_table
# df_Asian_second_table
# df_other_second_table

In [204]:
#thrid way to get data of client seen by age
col_names_seen_by_age_sheet1_2  <- c('Ethnic_group', 'Sex', 'total', age_group_col_names)
client_seen_by_age_data <- file_name[1] %>% read_excel(sheet = "table1&2", skip=2, range='A4:U19')
colnames(client_seen_by_age_data) <- col_names_seen_by_age_sheet1_2 


#thrid way to get data of client seen by rate
col_names_seen_by_Crude_rate_sheet1_2  <- c('Ethnic_group', 'Sex', 'Crude_rate', age_group_col_names, "ASR")
client_seen_by_Crude_rate_data <- file_name[1] %>% read_excel(sheet = "table1&2", skip=2, range='A26:V41')
colnames(client_seen_by_Crude_rate_data) <- col_names_seen_by_Crude_rate_sheet1_2 



#get num of row
n_row_age_sheet1_2 <- nrow(client_seen_by_age_data)

#set all na values with activity type 
for (row_num in 1:n_row_age_sheet1_2){
    if(client_seen_by_age_data$Ethnic_group[[row_num]] %>% is.na()){
        client_seen_by_age_data$Ethnic_group[[row_num]] = client_seen_by_age_data$Ethnic_group[[row_num - 1]]
    }
}


#get num of row
n_row_Crude_rate_sheet1_2 <- nrow(client_seen_by_Crude_rate_data)

#set all na values with activity type 
for (row_num in 1:n_row_Crude_rate_sheet1_2){
    if(client_seen_by_Crude_rate_data$Ethnic_group[[row_num]] %>% is.na()){
        client_seen_by_Crude_rate_data$Ethnic_group[[row_num]] = client_seen_by_Crude_rate_data$Ethnic_group[[row_num - 1]]
    }
}

#get ride of na value in ethnic_group col values
client_seen_by_age_data
client_seen_by_Crude_rate_data

New names:
* `` -> ...1
* `` -> ...2
* `` -> ...3

New names:
* `` -> ...1
* `` -> ...2
* `` -> ...3
* `` -> ...22



Ethnic_group,Sex,total,0-,5-,10-,15-,20-,25-,30-,⋯,40-,45-,50-,55-,60-,65-,70-,75-,80-,85+
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Total,Total,158158,1415,6764,14041,20401,16614,14168,12932,⋯,13133,11307,9707,6760,4636,3434,2817,2559,2417,2708
Total,Male,83267,920,4730,7135,10001,9345,7750,6855,⋯,7071,6088,5071,3507,2308,1546,1247,1091,961,990
Total,Female,74891,495,2034,6906,10400,7269,6418,6077,⋯,6062,5219,4636,3253,2328,1888,1570,1468,1456,1718
Māori,Total,40803,420,1778,4739,5923,5167,4448,3789,⋯,3561,2785,2095,1125,582,303,242,167,106,64
Māori,Male,23038,284,1281,2494,3338,3106,2534,2170,⋯,1961,1537,1072,565,298,144,110,73,38,22
Māori,Female,17765,136,497,2245,2585,2061,1914,1619,⋯,1600,1248,1023,560,284,159,132,94,68,42
Pacific,Total,8919,101,283,860,1406,1180,1048,891,⋯,699,565,386,247,147,122,81,87,43,22
Pacific,Male,5463,58,217,435,823,784,701,586,⋯,443,336,220,130,81,60,36,44,17,7
Pacific,Female,3456,43,66,425,583,396,347,305,⋯,256,229,166,117,66,62,45,43,26,15


Ethnic_group,Sex,Crude_rate,0-,5-,10-,15-,20-,25-,30-,⋯,45-,50-,55-,60-,65-,70-,75-,80-,85+,ASR
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Total,Total,3507.3,459.3,2203.6,4739.9,6512.8,5116.3,4837.6,4665.7,⋯,3630.2,3060.9,2420.9,1891.2,1590.0,1761.7,2249.0,2914.9,3503.0,3681.1
Total,Male,3769.7,583.0,3002.2,4706.2,6217.0,5632.1,5366.9,5147.2,⋯,4070.6,3309.4,2585.2,1932.0,1465.1,1616.2,2057.3,2627.1,3518.1,3970.3
Total,Female,3255.4,329.3,1361.4,4775.3,6825.0,4577.5,4322.8,4220.4,⋯,3223.4,2828.6,2265.6,1852.3,1709.4,1897.4,2416.3,3142.0,3494.4,3400.5
Māori,Total,5774.6,501.9,2223.9,6552.8,8442.1,8533.4,9445.7,9453.6,⋯,6962.5,5464.3,3720.2,2565.0,1894.9,2272.3,2565.3,2960.9,3535.9,6031.2
Māori,Male,6693.6,662.0,3114.5,6715.1,9290.3,10443.8,11645.2,11845.0,⋯,8153.8,5982.1,3990.1,2790.3,1917.4,2200.0,2543.6,2638.9,3492.1,7098.1
Māori,Female,4901.9,333.5,1280.3,6381.5,7551.9,6689.4,7556.3,7440.3,⋯,5900.7,5009.8,3482.6,2364.7,1875.0,2336.3,2582.4,3177.6,3559.3,5063.9
Pacific,Total,3064.4,329.8,975.9,3001.7,4891.3,4429.4,4916.7,4661.3,⋯,3189.4,2599.3,2142.2,1580.6,1798.1,1753.2,3047.3,2638.0,2430.9,3103.0
Pacific,Male,3798.6,371.0,1459.8,2968.3,5670.0,5991.6,6717.8,6274.1,⋯,3948.3,3076.9,2315.2,1782.2,1815.4,1702.1,3478.3,2698.4,2413.8,3871.6
Pacific,Female,2347.3,287.0,466.9,3036.8,4097.0,2921.4,3189.3,3120.2,⋯,2487.8,2155.8,1978.0,1388.0,1781.6,1796.4,2704.4,2600.0,2439.0,2363.7


In [192]:
#get col names for sheet 11
Ethnic_Group_names <- file_name[1] %>% read_excel(sheet = "table11", skip=2, range = cell_rows(4:4)) %>% colnames()
df_col_names_sheet_11 <- c("Organisation_name", Ethnic_Group_names)

#get df of sheet11
Ethnic_Group_sheet11_data <- file_name[1] %>% read_excel(sheet = "table11", skip=2, range='A4:G682')
colnames(Ethnic_Group_sheet11_data) <- df_col_names_sheet_11 
Ethnic_Group_sheet11_data

New names:
* `` -> ...1



Organisation_name,Sex,Total,Māori,Pacific,Asian,Other
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ACROSS - Te Kotahitanga o Te Wairua,Total,13,4,0,0,9
NA,Male,4,0,0,0,4
NA,Female,9,4,0,0,5
ARC Counselling Services,Total,498,252,7,2,237
NA,Male,317,167,7,0,143
NA,Female,181,85,0,2,94
Adventure Development Limited,Total,614,118,11,4,481
NA,Male,319,70,6,2,241
NA,Female,295,48,5,2,240


In [191]:
#set col names for sheet12 df
DHB_client_domicile_names_sheet12 <- file_name[1] %>% read_excel(sheet = "table12", skip=2, range = cell_rows(4:4)) %>% colnames()

#get df of sheet12
DHB_client_domicile_sheet12_df <- file_name[1] %>% read_excel(sheet = "table12", skip=2, range='A5:W26')
colnames(DHB_client_domicile_sheet12_df) <- DHB_client_domicile_names_sheet12 
DHB_client_domicile_sheet12_df

New names:
* `` -> ...1
* `` -> ...2
* `` -> ...3
* `` -> ...4
* `` -> ...5
* ...



DHB of service,Northland,Waitemata,Auckland,Counties Manukau,Waikato,Lakes,Bay,Tairawhiti,Taranaki,⋯,Hutt Valley,Wairarapa,Nelson Marlborough,West Coast,Canterbury,South Canterbury,Southern,Unknown,MidCentral,Unique Total
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Northland,5905,206,146,157,82,20,32,10,15,⋯,9,2,7,3,23,3,17,51,14,6356
Waitemata,731,15380,6660,5232,463,114,168,30,64,⋯,84,9,46,1,116,4,67,106,67,26087
Auckland,137,1137,9917,858,232,65,121,24,33,⋯,35,1,24,1,69,8,45,64,36,11654
Counties Manukau,150,394,1237,11473,207,60,67,12,28,⋯,100,1,12,2,63,2,23,58,21,12588
Waikato,67,114,135,172,9670,190,341,40,105,⋯,36,4,16,4,45,4,52,99,32,10547
Lakes,18,22,50,35,118,3643,142,9,12,⋯,10,2,4,1,12,2,36,32,25,3968
Bay of Plenty,22,52,63,63,160,78,7781,15,16,⋯,14,5,11,4,31,3,31,80,16,8034
Tairawhiti,3,12,4,18,20,8,12,2047,2,⋯,10,2,3,1,7,0,3,6,8,2142
Taranaki,9,17,25,17,21,10,14,2,3985,⋯,5,6,7,0,10,0,4,98,30,4088


In [190]:
#set col names for sheet17 df
client_ages_sheet17 <- file_name[1] %>% read_excel(sheet = "table17", skip=2, range = cell_rows(4:4)) %>% colnames()
client_Activity_names_sheet17 <- c('Activity_type', 'Sex', 'total',client_ages_sheet17)

#get df of sheet17
client_Activity_sheet17_df <- file_name[1] %>% read_excel(sheet = "table17", skip=2, range='A4:U130')
colnames(client_Activity_sheet17_df) <- client_Activity_names_sheet17

#get num of row
n_row_sheet17 <- nrow(client_Activity_sheet17_df)

#set all na values with activity type 
for (row_num in 1:n_row_sheet17){
    if(client_Activity_sheet17_df$Activity_type[[row_num]] %>% is.na()){
        client_Activity_sheet17_df$Activity_type[[row_num]] = client_Activity_sheet17_df$Activity_type[[row_num - 1]]
    }
}
client_Activity_sheet17_df

New names:
* `` -> ...1
* `` -> ...2
* `` -> ...3



Activity_type,Sex,total,0-,5-,10-,15-,20-,25-,30-,⋯,40-,45-,50-,55-,60-,65-,70-,75-,80-,85+
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Mental health crisis attendances,Total,38944,22,192,1901,5590,5280,4118,3544,⋯,3699,3261,2672,1884,1239,709,498,314,269,198
Mental health crisis attendances,Male,18836,11,135,602,2483,2797,2138,1810,⋯,1815,1603,1259,928,573,320,218,137,122,87
Mental health crisis attendances,Female,20108,11,57,1299,3107,2483,1980,1734,⋯,1884,1658,1413,956,666,389,280,177,147,111
Mental health intensive care inpatient or equivalent occupied bed nights,Total,3722,0,0,36,397,527,474,424,⋯,404,396,259,173,121,71,31,20,10,7
Mental health intensive care inpatient or equivalent occupied bed nights,Male,2220,0,0,17,260,363,316,255,⋯,232,209,131,91,63,29,17,12,5,5
Mental health intensive care inpatient or equivalent occupied bed nights,Female,1502,0,0,19,137,164,158,169,⋯,172,187,128,82,58,42,14,8,5,2
Mental health acute inpatient or equivalent occupied bed nights,Total,8632,4,19,180,894,966,867,769,⋯,835,787,626,469,353,304,242,204,199,164
Mental health acute inpatient or equivalent occupied bed nights,Male,4444,2,15,49,429,606,511,420,⋯,435,400,284,231,164,128,111,91,96,65
Mental health acute inpatient or equivalent occupied bed nights,Female,4188,2,4,131,465,360,356,349,⋯,400,387,342,238,189,176,131,113,103,99


In [189]:
#set col names for sheet21 df
number_team_DHB_sheet21 <- file_name[1] %>% read_excel(sheet = "table21", skip=2, range = cell_rows(4:4)) %>% colnames()
Team_type_DHB_names_sheet21 <- c('Team_type', number_team_DHB_sheet21)

#get df of sheet21
client_team_DHB_sheet21_df <- file_name[1] %>% read_excel(sheet = "table21", skip=2, range='A4:D17')
colnames(client_team_DHB_sheet21_df) <- Team_type_DHB_names_sheet21
client_team_DHB_sheet21_df

New names:
* `` -> ...1



Team_type,Clients seen,Bednights,Contacts
<chr>,<dbl>,<dbl>,<dbl>
Inpatient Team,9881,279531,13951
Community Team,103396,4160,2056470
Alcohol and Drug Team,29495,11343,426268
Forensic Team,5527,84615,59238
Residential/Accommodation Team,414,1607,3176
Co-existing Problems Team,1068,0,13300
Intellectual Disability Dual Diagnosis Team,739,7858,9915
Specialty Team,3976,4275,56586
Maternal Mental Health Team,2784,1743,37354


In [185]:
#set col names for sheet22 df
number_NGO_sheet22_name <- file_name[1] %>% read_excel(sheet = "table22", skip=2, range = cell_rows(4:4)) %>% colnames()
Team_type_client_team_NGO_sheet22_name <- c('Team_type', number_NGO_sheet22_name)

#get df of sheet22
client_team_NGO_sheet22_df <- file_name[1] %>% read_excel(sheet = "table22", skip=2, range='A4:D17')
colnames(client_team_NGO_sheet22_df) <- Team_type_client_team_NGO_sheet22_name
client_team_NGO_sheet22_df

New names:
* `` -> ...1



Team_type,Clients seen,Bednights,Contacts
<chr>,<dbl>,<dbl>,<dbl>
Inpatient Team,8,2642,0
Community Team,33427,20065,1295047
Alcohol and Drug Team,20825,140308,246394
Forensic Team,47,8499,3946
Residential/Accommodation Team,6051,543178,33283
Co-existing Problems Team,715,13016,8754
Intellectual Disability Dual Diagnosis Team,2,730,0
Specialty Team,1032,2240,16431
Maternal Mental Health Team,115,0,2028
